In [4]:
import graphlab as gl
import matplotlib.pyplot as plt
import os
import graphlab.aggregate as agg
import numpy as np
from scipy.stats import gaussian_kde
import seaborn as sns
import math

In [76]:
 
def Edges(P):
    points = sorted(set(P))
    def cross(o, a, b):
        return (a[0] - o[0]) * (b[1] - o[1]) - (a[1] - o[1]) * (b[0] - o[0])
    lower = []
    for count in points:
            while len(lower) >= 2 and cross(lower[-2], lower[-1], count) <= 0:
                lower.pop()
            lower.append(count)
    upper = []
    for count in reversed(points):
            while len(upper) >= 2 and cross(upper[-2], upper[-1], count) <= 0:
                upper.pop()
            upper.append(count)
    shape = lower[:-1] + upper[:-1]
    return shape
#Finds the Area from the edges verticies through the shoelace algorithm
def Area(shape):
    Area = 0
    Add = 0
    Sub = 0
    for a in range(len(shape)):
        if a != (len(shape)-1):
            Add = Add + shape[a][0]* shape[a+1][1]
            Sub = Sub + shape[a][1]* shape[a+1][0]
        else:
            Add = Add + shape[a][0]* shape[0][1]
            Sub = Sub + shape[a][1]* shape[0][0]
    Area = (((Add - Sub)**2)**0.5)* 0.5
    return Area

def Rock_and_Roll(filename): 

   
    
    #import in the txt file as a SFrame
    Raw_Image = gl.SFrame.read_csv(filename, delimiter = '	',header = True)
     
    #shift points so the min is (0,0)
    Sauce = gl.SFrame({'X': Raw_Image['Xc']-min(Raw_Image['Xc']),'Y': Raw_Image['Yc']-min(Raw_Image['Yc'])})
    
    #Runs DBSCAN on the points
    #Cluster
    Image_C = gl.dbscan.create(Sauce, radius = 30, min_core_neighbors = 30)
    Noise_Filter = gl.dbscan.create(Sauce, radius = 30, min_core_neighbors = 2)
    Sauce = Sauce.add_row_number('row_id')
    
    #Add the cluster identity to each point 
    Image_Points = Sauce.join(Image_C['cluster_id'], on='row_id', how='left')
    Noise_Filter_p = Sauce.join(Noise_Filter['cluster_id'], on='row_id', how='left')  
    
    #Removes non clustered points
    Image_Pts = Image_Points[Image_Points['cluster_id']>=0]
    Noise_Filter_p = Noise_Filter_p[Noise_Filter_p['cluster_id']>=0]
    
    #Determines subject and treatment from filename
    if filename.find('++') > 0:
        #Treat = 'Antagonist'
        Treat = 'CytD'
    elif filename.find('+-')>0:
        #Treat = 'NMDA'
        #Treat = 'Nicotine'
        #Treat = 'KCl'
        Treat = 'mbCD'
    else :
        Treat = "Control"
    if filename.find('E')>0:
        Subject= 'Extension'
    elif filename.find('V')>0:
        Subject ='Varicosity' 
    else:
        Subject ='Soma'
        
    #Creates SArray to store cluster volumes in    
    Clus_Vol = gl.SArray(data=[])
        
    #Initialize total clustered area of subject to be 0
    Area_t = 0
    Total_Clusters = Image_Pts.groupby(key_columns='cluster_id',operations={'Num_points':agg.COUNT('row_id')})
    Total_Clusters = Total_Clusters.sort('cluster_id')
    
    #Determines the areas of each cluster and total clustered area of the subject
    Image_Points = Image_Points.sort('cluster_id', ascending = False)
    print Image_Points[0]['cluster_id']
    if Image_Points[0]['cluster_id'] != None:
        #The number of clusters present in the subject
        Num_C = int((Image_Pts.sort('cluster_id', ascending = False))[0]['cluster_id'])
        for c in range(0,Num_C+1):
            cluster_mask = Image_Pts['cluster_id'] == c
            x,y = Image_Pts['X'][cluster_mask],Image_Pts['Y'][cluster_mask]
            P = zip(x,y)
            shape_c = Edges(P)
            Area_c = Area(shape_c)
            Area_t = Area_c + Area_t
            qwe = gl.SArray(data=[Area_c])
            Clus_Vol = Clus_Vol.append(qwe)
        Total_Clusters = Total_Clusters.add_column(Clus_Vol,name ='Cluster Area')
        Total_Clusters = Total_Clusters.add_column(Total_Clusters['Num_points']/Total_Clusters['Cluster Area'],name='Cluster Density')
    else:
        Num_C = -1
        Total_Clusters = gl.SFrame({'cluster_id':[0],'Num_points':[0],
                   'Cluster Area':[0.0],'Cluster Density':[0.0]})
 
    Total_Clusters['Treatment']=Treat
    Total_Clusters['Subject']=Subject
    Total_Clusters['File']=filename
     
    Image_Data = Total_Clusters.groupby(key_columns='File', operations ={'Mean Cluster Area':agg.MEAN('Cluster Area'),
                                                                         'Mean Num Pts in Clus':agg.MEAN('Num_points'),
                                                                        'Mean Density':agg.MEAN('Cluster Density')})
    Image_Data['Treatment']=Treat
    Image_Data['Subject']=Subject
    Image_Data['Number of Clusters'] = Num_C+1
    Image_Data['Percent Pts clusters'] = float(Image_Pts.num_rows()) / float(Noise_Filter_p.num_rows())*100
    return (Total_Clusters,Image_Data)


In [74]:
Total_Clusters = gl.SFrame({'cluster_id':gl.SArray(dtype = int),'Num_points':gl.SArray(dtype = int),
                   'Cluster Area':gl.SArray(dtype = float),
                   'Cluster Density':gl.SArray(dtype = float),'Treatment':gl.SArray(dtype = str), 'File':gl.SArray(dtype = str),
                   'Subject':gl.SArray(dtype = str)})
Total_Image =gl.SFrame({'Treatment':gl.SArray(dtype = str), 'File':gl.SArray(dtype = str),
                   'Subject':gl.SArray(dtype = str),'Number of Clusters':gl.SArray(dtype = int),
                       'Percent Pts clusters':gl.SArray(dtype = float),
                       'Mean Cluster Area':gl.SArray(dtype = float),'Mean Density':gl.SArray(dtype = float),
                       'Mean Num Pts in Clus':gl.SArray(dtype = float)})
for f in os.listdir('Cluster_Data'):     #loads up every txt file in the cluster_data dir but sources from main dir
    C,I = Rock_and_Roll(f)
    Total_Clusters = Total_Clusters.append(C) #makes master list
    Total_Image = Total_Image.append(I)  #makes master list
    Total_Clusters = Total_Clusters.sort(['Subject',"Treatment",'File','cluster_id'])
    Total_Image = Total_Image.sort(['Subject',"Treatment",'File'])
    Total_Clusters.export_csv(filename = 'Total Clusters.txt', delimiter = ' 	')
    Total_Image.export_csv(filename = 'Total Image.txt', delimiter = ' 	')

Finished parsing file C:\Users\jdr248\130919 -- E 001.txt

Parsing completed. Parsed 100 lines in 0.034003 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,float,float,float,float,float,float,float,long,long,long,long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\jdr248\130919 -- E 001.txt

Parsing completed. Parsed 20780 lines in 0.062206 secs.

...

| 2732                        |

| 0                           |

+-----------------------------+

[INFO] graphlab.toolkits.clustering.dbscan: Processing boundary points.


0


In [1]:
Raw_Data = gl.SFrame.read_csv('Total Clusters.txt', delimiter = '	',header = True)
Dataframe = Raw_Data.to_dataframe()
pal = sns.cubehelix_palette(8, rot=-.5, dark=.3)
plt.ylim(0, 200)
sns.violinplot(x='Treatment',y ='Num_points', hue="Subject",order =['Control','Nicotine','Antagonist'], data=Dataframe, split=True,
               inner="quart", palette={"Extension": "b", "Varicosity": "r"})
plt.show()

NameError: name 'gl' is not defined

In [17]:
Raw_Data = gl.SFrame.read_csv('160519 -- V 001.txt', delimiter = '	',header = True)
Sauce = gl.SFrame({'X': Raw_Data['Xc']-min(Raw_Data['Xc']),'Y': Raw_Data['Yc']-min(Raw_Data['Yc'])})
Dataframe = Sauce.to_dataframe()
#sns.kdeplot(Dataframe['X'],Dataframe['Y'],cmap="Reds",bw=60, shade=True, shade_lowest=False)
#sns.jointplot(x='X',y= 'Y', data=Dataframe, kind="kde",bw=90,cmap="Reds",shade_lowest=False,shade=True)
g = sns.jointplot(x='X',y= 'Y', data=Dataframe, kind="kde",bw=90,cmap="Reds",shade_lowest=False,shade=True)
g.plot_joint(plt.scatter, c="b", s=10, linewidth=1, marker="+")
plt.show()

Finished parsing file C:\Users\jdr248\160519 -- V 001.txt

Parsing completed. Parsed 100 lines in 0.039004 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,str,str,str,str,float,str,str,long,long,long,long,long,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\jdr248\160519 -- V 001.txt

Parsing completed. Parsed 8402 lines in 0.028003 secs.

In [5]:
plt.close()
Raw_Data = gl.SFrame.read_csv('Total Clusters.txt', delimiter = '	',header = True)
Dataframe = Raw_Data.to_dataframe()
VD = Dataframe.loc[Dataframe['Subject'] == 'Varicosity']
VCD = VD.loc[VD['Treatment']=='Control']
VND = VD.loc[VD['Treatment']=='Nicotine']
VAD = VD.loc[VD['Treatment']=='Antagonist']
sns.distplot(VCD['Num_points'],color = 'red', hist = False,label="Control");
sns.distplot(VND['Num_points'],color = 'blue', hist = False,label="Nicotine");
sns.distplot(VAD['Num_points'],color = 'yellow', hist = False,label="Antagonist");
plt.xlim(0, 5000)
plt.legend()
plt.show()

Finished parsing file C:\Users\jdr248\Total Clusters.txt

Parsing completed. Parsed 100 lines in 0.020002 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,str,long,str,str,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\jdr248\Total Clusters.txt

Parsing completed. Parsed 3977 lines in 0.017002 secs.

In [20]:
plt.close()
sns.set(style="white", palette="muted", color_codes=True)
f, axes = plt.subplots(3, 3, figsize=(10, 10), sharex=True)
sns.despine(left=False)
Raw_Data = gl.SFrame.read_csv('Total Clusters Nicotine.txt', delimiter = '	',header = True)
Dataframe = Raw_Data.to_dataframe()
VD = Dataframe.loc[Dataframe['Subject'] == 'Extension']
VCD = VD.loc[VD['Treatment']=='Control']
VND = VD.loc[VD['Treatment']=='Nicotine']
VAD = VD.loc[VD['Treatment']=='Antagonist']
#sns.distplot(VCD['Cluster Area'],color = 'red', hist = False,label="Control");
Delts = 2000
sns.distplot(VCD['Cluster Area'], bins=np.arange(0, max(VCD['Cluster Area']) + Delts, Delts), color="b", ax=axes[0, 0])
sns.distplot(VND['Cluster Area'], bins=np.arange(0, max(VND['Cluster Area']) + Delts, Delts), color="r", ax=axes[1, 1])
sns.distplot(VAD['Cluster Area'], bins=np.arange(0, max(VAD['Cluster Area']) + Delts, Delts), color="g", ax=axes[2, 2])

sns.distplot(VCD['Cluster Area'], bins=np.arange(0, max(VCD['Cluster Area']) + Delts, Delts), color="b", ax=axes[0, 1])
sns.distplot(VND['Cluster Area'], bins=np.arange(0, max(VND['Cluster Area']) + Delts, Delts), color="r", ax=axes[0, 1])

sns.distplot(VND['Cluster Area'], bins=np.arange(0, max(VND['Cluster Area']) + Delts, Delts), color="r", ax=axes[1, 0])
sns.distplot(VCD['Cluster Area'], bins=np.arange(0, max(VCD['Cluster Area']) + Delts, Delts), color="b", ax=axes[1, 0])

sns.distplot(VCD['Cluster Area'], bins=np.arange(0, max(VCD['Cluster Area']) + Delts, Delts), color="b", ax=axes[0, 2])
sns.distplot(VAD['Cluster Area'], bins=np.arange(0, max(VAD['Cluster Area']) + Delts, Delts), color="g", ax=axes[0, 2])

sns.distplot(VAD['Cluster Area'], bins=np.arange(0, max(VAD['Cluster Area']) + Delts, Delts), color="g", ax=axes[2, 0])
sns.distplot(VCD['Cluster Area'], bins=np.arange(0, max(VCD['Cluster Area']) + Delts, Delts), color="b", ax=axes[2, 0])

sns.distplot(VND['Cluster Area'], bins=np.arange(0, max(VND['Cluster Area']) + Delts, Delts), color="r", ax=axes[1, 2])
sns.distplot(VAD['Cluster Area'], bins=np.arange(0, max(VAD['Cluster Area']) + Delts, Delts), color="g", ax=axes[1, 2])

sns.distplot(VAD['Cluster Area'], bins=np.arange(0, max(VAD['Cluster Area']) + Delts, Delts), color="g", ax=axes[2, 1])
sns.distplot(VND['Cluster Area'], bins=np.arange(0, max(VND['Cluster Area']) + Delts, Delts), color="r", ax=axes[2, 1])


plt.xlim(10000, 500000)
plt.setp(axes, yticks=[])
plt.tight_layout()
plt.show()

Finished parsing file C:\Users\jdr248\Total Clusters.txt

Parsing completed. Parsed 100 lines in 0.017002 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,str,long,str,str,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\jdr248\Total Clusters.txt

Parsing completed. Parsed 3977 lines in 0.017002 secs.

In [47]:
plt.close()
sns.set(style="white", palette="muted", color_codes=True)
f, axes = plt.subplots(3, 3, figsize=(10, 10), sharex=True,sharey=True)
#sns.despine(left=False)
Raw_Data = gl.SFrame.read_csv('Total Clusters Nicotine.txt', delimiter = '	',header = True)
Dataframe = Raw_Data.to_dataframe()
Clean = Dataframe.loc[Dataframe['Cluster Area'] < 50000]
Clean = Dataframe.loc[Dataframe['Cluster Area'] > 0]
SF= gl.SFrame(data= Clean)
SF['Log of Area'] = SF.apply(lambda x:math.log(float(x['Cluster Area']),10))
Dataframe = SF.to_dataframe()
VD = Dataframe.loc[Dataframe['Subject'] == 'Extension']
VCD = VD.loc[VD['Treatment']=='Control']
VND = VD.loc[VD['Treatment']=='Nicotine']
VAD = VD.loc[VD['Treatment']=='Antagonist']
#sns.distplot(VCD['Cluster Area'],color = 'red', hist = False,label="Control");
Delts = .1
Column = 'Log of Area'
Colorc = 'blue'
Colorn = 'red'
Colora = 'green'
axes[0,0].set_title('Control')
sns.distplot(VCD[Column], bins=np.arange(0, max(VCD[Column]) + Delts, Delts),hist=False,  color=Colorc, ax=axes[0, 0])
axes[1,1].set_title('Nicotine')
sns.distplot(VND[Column], bins=np.arange(0, max(VND[Column]) + Delts, Delts),hist=False,  color=Colorn, ax=axes[1, 1])
axes[2,2].set_title('Antagonist')
sns.distplot(VAD[Column], bins=np.arange(0, max(VAD[Column]) + Delts, Delts),hist=False,  color=Colora, ax=axes[2, 2])
axes[0,1].set_title('Nicotine vs Control')
sns.distplot(VCD[Column], bins=np.arange(0, max(VCD[Column]) + Delts, Delts),hist=False,color=Colorc, ax=axes[0, 1])
sns.distplot(VND[Column], bins=np.arange(0, max(VND[Column]) + Delts, Delts),hist=False, color=Colorn, ax=axes[0, 1])
axes[1,0].set_title('Control vs Nicotine')
sns.distplot(VND[Column], bins=np.arange(0, max(VND[Column]) + Delts, Delts),hist=False, color=Colorn, ax=axes[1, 0])
sns.distplot(VCD[Column], bins=np.arange(0, max(VCD[Column]) + Delts, Delts),hist=False, color=Colorc, ax=axes[1, 0])
axes[0,2].set_title('Antagonist vs Control')
sns.distplot(VCD[Column], bins=np.arange(0, max(VCD[Column]) + Delts, Delts),hist=False, color=Colorc, ax=axes[0, 2])
sns.distplot(VAD[Column], bins=np.arange(0, max(VAD[Column]) + Delts, Delts),hist=False, color=Colora, ax=axes[0, 2])
axes[2,0].set_title('Control vs Antagonist')
sns.distplot(VAD[Column], bins=np.arange(0, max(VAD[Column]) + Delts, Delts),hist=False, color=Colora, ax=axes[2, 0])
sns.distplot(VCD[Column], bins=np.arange(0, max(VCD[Column]) + Delts, Delts),hist=False, color=Colorc, ax=axes[2, 0])
axes[1,2].set_title('Antagonist vs Nicotine')
sns.distplot(VND[Column], bins=np.arange(0, max(VND[Column]) + Delts, Delts),hist=False, color=Colorn, ax=axes[1, 2])
sns.distplot(VAD[Column], bins=np.arange(0, max(VAD[Column]) + Delts, Delts),hist=False, color=Colora, ax=axes[1, 2])
axes[2,1].set_title('Nicotine vs Antagonist')
sns.distplot(VAD[Column], bins=np.arange(0, max(VAD[Column]) + Delts, Delts),hist=False, color=Colora, ax=axes[2, 1])
sns.distplot(VND[Column], bins=np.arange(0, max(VND[Column]) + Delts, Delts),hist=False, color=Colorn, ax=axes[2, 1])

axes[0,0].set(xlabel='Log of Area', ylabel='Frequency')
axes[1,1].set(xlabel='Log of Area', ylabel='Frequency')
axes[2,2].set(xlabel='Log of Area', ylabel='Frequency')
axes[0,1].set(xlabel='Log of Area', ylabel='Frequency')
axes[1,0].set(xlabel='Log of Area', ylabel='Frequency')
axes[0,2].set(xlabel='Log of Area', ylabel='Frequency')
axes[2,0].set(xlabel='Log of Area', ylabel='Frequency')
axes[1,2].set(xlabel='Log of Area', ylabel='Frequency')
axes[2,1].set(xlabel='Log of Area', ylabel='Frequency')


#plt.xlim(4,7)
plt.setp(axes, yticks=[])
plt.tight_layout()
plt.show()

Finished parsing file C:\Users\jdr248\Total Clusters Nicotine.txt

Parsing completed. Parsed 100 lines in 0.024002 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,str,long,str,str,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\jdr248\Total Clusters Nicotine.txt

Parsing completed. Parsed 3977 lines in 0.016002 secs.

In [42]:
plt.close()
sns.set(style="white", palette="muted", color_codes=True)
sns.despine(left=False)
Raw_Data = gl.SFrame.read_csv('Total Clusters.txt', delimiter = '	',header = True)
Dataframe = Raw_Data.to_dataframe()
Clean = Dataframe.loc[Dataframe['Cluster Area'] < 50000]
Clean = Dataframe.loc[Dataframe['Cluster Area'] > 0]
SF= gl.SFrame(data= Clean)
SF['Log of Area'] = SF.apply(lambda x:math.log(float(x['Cluster Area']),10))
Dataframe = SF.to_dataframe()
VD = Dataframe.loc[Dataframe['Subject'] == 'Varicosity']

VND = VD.loc[VD['Treatment']=='NMDA']
Delts = 10000
sns.distplot(VND['Cluster Area'],bins=np.arange(0, max(VND['Cluster Area']) + Delts, Delts),kde = False, rug = True,color = 'red', label="NMDA");

#plt.xlim(10000,570000)
#plt.ylim(0,1)
plt.setp(axes, yticks=[])
plt.tight_layout()
plt.show()

Finished parsing file C:\Users\jdr248\Total Clusters.txt

Parsing completed. Parsed 100 lines in 0.017002 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,str,long,str,str,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\jdr248\Total Clusters.txt

Parsing completed. Parsed 4139 lines in 0.015001 secs.

In [41]:
import math
plt.close()
sns.set(style="white", palette="muted", color_codes=True)
Raw_Data = gl.SFrame.read_csv('Total Clusters.txt', delimiter = '	',header = True)

Dataframe = Raw_Data.to_dataframe()
Clean = Dataframe.loc[Dataframe['Cluster Area'] < 50000]
Clean = Dataframe.loc[Dataframe['Cluster Area'] > 0]
SF= gl.SFrame(data= Clean)
SF['Log of Area'] = SF.apply(lambda x:math.log(float(x['Cluster Area']),10))
Dataframe = SF.to_dataframe()
VD = Dataframe.loc[Dataframe['Subject'] == 'Varicosity']
VCD = VD.loc[VD['Treatment']=='Control']
VND = VD.loc[VD['Treatment']=='NMDA']
VAD = VD.loc[VD['Treatment']=='Antagonist']

Delts = 0.2

f, axarr = plt.subplots(3, 3,sharex=True,sharey=True)
axarr[0, 0].set_title('Control')
axarr[0, 0].hist(VCD['Log of Area'],bins=np.arange(0, max(VCD['Log of Area']) + Delts, Delts),color = 'blue', label="Control")
axarr[1, 1].set_title('NMDA')
axarr[1, 1].hist(VND['Log of Area'],bins=np.arange(0, max(VND['Log of Area']) + Delts, Delts),color = 'red', label="NMDA")
axarr[2, 2].set_title('Antagonist')
axarr[2, 2].hist(VAD['Log of Area'],bins=np.arange(0, max(VAD['Log of Area']) + Delts, Delts),color = 'green', label="Antagonist")

axarr[1, 0].set_title('Control vs NMDA')
axarr[1, 0].hist(VND['Log of Area'],bins=np.arange(0, max(VND['Log of Area']) + Delts, Delts),color = 'red', label="NMDA")
axarr[1, 0].hist(VCD['Log of Area'],bins=np.arange(0, max(VCD['Log of Area']) + Delts, Delts),color = 'blue', label="Control")

axarr[2, 0].set_title('Control vs Antagonist')
axarr[2, 0].hist(VAD['Log of Area'],bins=np.arange(0, max(VAD['Log of Area']) + Delts, Delts),color = 'green', label="Antagonist")
axarr[2, 0].hist(VCD['Log of Area'],bins=np.arange(0, max(VCD['Log of Area']) + Delts, Delts),color = 'blue', label="Control")

axarr[0, 1].set_title('NMDA vs Control')
axarr[0, 1].hist(VCD['Log of Area'],bins=np.arange(0, max(VCD['Log of Area']) + Delts, Delts),color = 'blue', label="Control")
axarr[0, 1].hist(VND['Log of Area'],bins=np.arange(0, max(VND['Log of Area']) + Delts, Delts),color = 'red', label="NMDA")

axarr[2, 1].set_title('NMDA vs Antagonist')
axarr[2, 1].hist(VAD['Log of Area'],bins=np.arange(0, max(VAD['Log of Area']) + Delts, Delts),color = 'green', label="Antagonist")
axarr[2, 1].hist(VND['Log of Area'],bins=np.arange(0, max(VND['Log of Area']) + Delts, Delts),color = 'red', label="NMDA")

axarr[0, 2].set_title('Antagonist vs Control')
axarr[0, 2].hist(VCD['Log of Area'],bins=np.arange(0, max(VCD['Log of Area']) + Delts, Delts),color = 'blue', label="Control")
axarr[0, 2].hist(VAD['Log of Area'],bins=np.arange(0, max(VAD['Log of Area']) + Delts, Delts),color = 'green', label="Antagonist")

axarr[1, 2].set_title('Antagonist vs NMDA')
axarr[1, 2].hist(VND['Log of Area'],bins=np.arange(0, max(VND['Log of Area']) + Delts, Delts),color = 'red', label="NMDA")
axarr[1, 2].hist(VAD['Log of Area'],bins=np.arange(0, max(VAD['Log of Area']) + Delts, Delts),color = 'green', label="Antagonist")

plt.xlabel('common xlabel')
plt.ylabel('common ylabel')
plt.setp(axes, yticks=[])
plt.tight_layout()
plt.show()

Finished parsing file C:\Users\jdr248\Total Clusters.txt

Parsing completed. Parsed 100 lines in 0.019002 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,str,long,str,str,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\jdr248\Total Clusters.txt

Parsing completed. Parsed 4139 lines in 0.016002 secs.

In [21]:
Raw_Data = gl.SFrame.read_csv('Total Clusters.txt', delimiter = '	',header = True)
Dataframe = Raw_Data.to_dataframe()
Clean = Dataframe.loc[Dataframe['Cluster Area'] < 50000]
Clean = Dataframe.loc[Dataframe['Cluster Area'] > 0]


Finished parsing file C:\Users\jdr248\Total Clusters.txt

Parsing completed. Parsed 100 lines in 0.017002 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,str,long,str,str,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\jdr248\Total Clusters.txt

Parsing completed. Parsed 3977 lines in 0.014001 secs.

In [31]:
plt.close()
sns.set(style="white", palette="muted", color_codes=True)
f, axes = plt.subplots(1 )
Raw_Data = gl.SFrame.read_csv('Total Clusters Nicotine.txt', delimiter = '	',header = True)
Dataframe = Raw_Data.to_dataframe()

Clean = Dataframe.loc[Dataframe['Cluster Area'] < 15000]
Clean = Clean.loc[Clean['Cluster Area'] > 0]

SF= gl.SFrame(data= Clean)

SF['Log of Area'] = SF.apply(lambda x:math.log(float(x['Cluster Area']),10))
Dataframe = SF.to_dataframe()
VD = Dataframe.loc[Dataframe['Subject'] == 'Extension']
VCD = VD.loc[VD['Treatment']=='Control']
VND = VD.loc[VD['Treatment']=='Nicotine']
VAD = VD.loc[VD['Treatment']=='Antagonist']
Delts = .1
Column = 'Log of Area'
Colorc = 'blue'
Colorn = 'red'
Colora = 'green'
sns.distplot(VCD[Column], bins=np.arange(0, max(VCD[Column]) + Delts, Delts), kde_kws={"bw": .1}, hist=False,  color=Colorc)
sns.distplot(VND[Column], bins=np.arange(0, max(VCD[Column]) + Delts, Delts), kde_kws={"bw": .1},hist=False,  color=Colorn)
sns.distplot(VAD[Column], bins=np.arange(0, max(VCD[Column]) + Delts, Delts), kde_kws={"bw": .1},hist=False,  color=Colora)
plt.setp(axes, yticks=[])
plt.tight_layout()
plt.xlim(2,5)
plt.show()

Finished parsing file C:\Users\jdr248\Total Clusters Nicotine.txt

Parsing completed. Parsed 100 lines in 0.019002 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,str,long,str,str,long]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\jdr248\Total Clusters Nicotine.txt

Parsing completed. Parsed 3977 lines in 0.016002 secs.